In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gensim
import gensim.models.keyedvectors as word2vec
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,KFold
from pandarallel import pandarallel

In [3]:
pandarallel.initialize(progress_bar=True,nb_workers=15,shm_size_mb=2500)


New pandarallel memory created - Size: 2500 MB
Pandarallel will run on 15 workers


In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
baseline_f = pd.read_csv('./www2018-table/feature/features.csv')

In [6]:
semantic_f_ = pd.read_csv('./semantic_f_w2v_g2v_upd_boe_boc.csv')

In [16]:
x_bf = ['row', 'col', 'nul', 'in_link', 'out_link', 'pgcount', 'tImp', 'tPF', 'leftColhits', 'SecColhits', 'bodyhits', 'PMI', 'qInPgTitle', 'qInTableTitle', 'yRank', 'csr_score', 'idf1', 'idf2', 'idf3', 'idf4', 'idf5', 'idf6', 'max', 'sum', 'avg', 'sim', 'emax', 'esum', 'eavg', 'esim', 'cmax', 'csum', 'cavg', 'csim', 'remax', 'resum', 'reavg', 'resim', 'query_l']
x_smf_w = ['w2v_early_fusion', 'w2v_late_fusion_max', 'w2v_late_fusion_avg', 'w2v_late_fusion_sum']
x_smf_g = ['g2v_early_fusion', 'g2v_late_fusion_max', 'g2v_late_fusion_avg', 'g2v_late_fusion_sum']
x_smf_boe = ['boe_early_fusion', 'boe_late_fusion_max', 'boe_late_fusion_avg', 'boe_late_fusion_sum']
x_smf_boc = ['boc_early_fusion', 'boc_late_fusion_max', 'boc_late_fusion_avg', 'boc_late_fusion_sum']
x_smf = x_smf_boe + x_smf_boc + x_smf_w + x_smf_g 
y_f = ['rel']

In [21]:
X = baseline_f[x_bf]
# X = pd.concat([baseline_f[x_bf],semantic_f_[x_smf]],axis=1)
y = baseline_f[y_f]

In [22]:
# d1 = pd.read_csv('./www2018-table/rankings/LTR.txt', sep="\t",header=None)
# d1.drop(columns=[1,3,4,5],inplace=True)

# l1 = [(row._1,row._2) for row in d1.itertuples()]
# ltr_in = []
# for row in baseline_f.itertuples():
#     if (row.query_id,row.table_id) in l1:
#         ltr_in.append(row.Index)
# anti_ltr_in = list(set(baseline_f.index.tolist()) - set(ltr_in))

In [23]:
X.head()

,row,col,nul,in_link,out_link,pgcount,tImp,tPF,leftColhits,SecColhits,bodyhits,PMI,qInPgTitle,qInTableTitle,yRank,csr_score,idf1,idf2,idf3,idf4,idf5,idf6,max,sum,avg,sim,emax,esum,eavg,esim,cmax,csum,cavg,csim,remax,resum,reavg,resim,query_l
0,8,2,0,31,21,51438,1.0,0.000260,0,0,0,0.000000,0.333333,0.222222,100,7.467415e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,9.266177,0.110312,0.676819,0.952804,89.707105,0.815519,0.971854,0.666667,5.291894,0.048108,0.354686,0.241209,3.716354,0.033785,0.281130,4
1,4,3,0,18,0,324,1.0,0.000786,0,0,0,-0.231049,0.000000,0.285714,100,8.663725e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,3.038269,0.108510,0.863125,1.000000,83.375200,0.757956,0.983893,1.000000,11.116121,0.101056,0.718895,1.000000,8.075247,0.073411,0.710250,4
2,3,5,0,23,22,26419,0.5,0.000158,0,0,0,-0.069315,0.000000,0.000000,100,2.142316e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,0.234136,1.747577,0.043689,0.206247,0.841653,57.442424,0.638249,0.753198,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4
3,4,5,1,23,22,26419,0.5,0.000211,0,0,0,-0.069315,0.000000,0.000000,100,1.888429e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,0.234136,1.641433,0.041036,0.252283,0.841653,57.442424,0.638249,0.753198,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4
4,2,2,0,38,1,2268,1.0,0.000130,0,0,0,0.000000,0.000000,0.500000,100,7.673589e-10,29.627861,24.135644,27.100638,31.219261,27.359181,27.100638,1.000000,4.982802,0.113245,0.700773,0.932809,89.552347,0.814112,0.954395,1.000000,10.147388,0.092249,0.372667,0.226134,4.564622,0.041497,0.279899,4


In [24]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# kfold = KFold(5, True, 42)
# for i, indices in enumerate(kfold.split(X_train)):
#     train, test = indices
#     X_train1, y_train1 = X_train.iloc[train], y_train.iloc[train]
#     clf.fit(X_train1, y_train1)
    
# X_test = generateScore(X_test)
# df = generate_sorted_df(X_test,y_test)
# df1 = generate_trec_df(df)
# df1.to_csv('./Results/LTR_tr80_5fld_tr20.txt', sep=' ', index=False, header=False)

kfold = KFold(5, True, 42)
for i, indices in enumerate(kfold.split(X)):
    train, test = indices
    X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
    df1 = makeModel_getdf(X_train, X_test, y_train, y_test)
    df1.to_csv('./Results/LTR_k5_{}.txt'.format(i), sep=' ', index=False, header=False)
    
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# X_test = pd.DataFrame()
# for i in range(1,61):
#     X_test = pd.concat([X_test,baseline_f[baseline_f.query_id==i][x_bf].sample(20)])
# y_test = baseline_f.iloc[X_test.index.values][y_f]
# y_train = baseline_f.iloc[list(set(baseline_f.index.values) - set(X_test.index.values))][y_f]
# X_train = baseline_f.iloc[list(set(baseline_f.index.values) - set(X_test.index.values))][x_bf]

# X_train = X.iloc[anti_ltr_in]
# X_test = X.iloc[ltr_in]
# y_train = y.iloc[anti_ltr_in]
# y_test = y.iloc[ltr_in]

# df1 = makeModel_getdf(X_train, X_test, y_train, y_test)
# df1.to_csv('./Results/test.txt', sep=' ', index=False, header=False)

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we ca

In [ ]:
!./trec_eval/trec_eval -m ndcg_cut.5,10,15,20 ./Results/qrels.txt ./Results/LTR_boe_k5_diff_0.txt

In [25]:
def makeModel_getdf(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=1000,max_features=3,random_state=42)
    clf.fit(X_train,y_train)
    X_test['model_score'] = X_test.parallel_apply(lambda x: getScore(x,clf),axis=1)
    df = generate_sorted_df(X_test,y_test)
    df1 = generate_trec_df(df)
    return df1

In [12]:
def getScore(row,clf):
    arr = clf.predict_proba(np.array(row).reshape(1,-1))
    return arr[0][1]+2*arr[0][2]

In [13]:
def generate_sorted_df(X,y):
#     df_temp = pd.concat([
#         X[['model_score']],
#         y],axis=1)
    df_temp = pd.concat([
        baseline_f.iloc[list(X.index)][['query_id', 'query', 'table_id']],
        X['model_score']],axis=1)

#     df_sorted = df_temp.sort_values(by=['query_id', 'model_score'], ascending=[True,False])

#     df1 = pd.DataFrame()
#     for i in range(1,61):
# #         print(i,df_sorted[df_sorted.query_id == i][:20].shape)
#         df1 = pd.concat([df1,df_sorted[df_sorted.query_id == i][:20]])
    return df_temp

In [14]:
def generate_trec_df(df1):
    l = []
    dic = dict(df1.query_id.value_counts())
    for i in dic:
        for j in range(1,dic[i]+1):
            l.append(j)
#     for i in range(60):
#         for j in range(1,21):
#             l.append(j)
    df2 = pd.DataFrame()
    df2['query_id'] = df1['query_id']
    df2['Q0'] = 'Q0'
    df2['table_id'] = df1['table_id']
    df2['rank'] = l
    df2['score'] = df1['model_score']
    df2['smarttable'] = 'smarttable'
    return df2